In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/creditcard-shiva/creditcard.csv


In [2]:
credit_data=pd.read_csv("/kaggle/input/creditcard-shiva/creditcard.csv")

In [3]:
#Print first 5 rows of dataframe
credit_data.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [4]:
credit_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284807 entries, 0 to 284806
Data columns (total 31 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Time    284807 non-null  float64
 1   V1      284807 non-null  float64
 2   V2      284807 non-null  float64
 3   V3      284807 non-null  float64
 4   V4      284807 non-null  float64
 5   V5      284807 non-null  float64
 6   V6      284807 non-null  float64
 7   V7      284807 non-null  float64
 8   V8      284807 non-null  float64
 9   V9      284807 non-null  float64
 10  V10     284807 non-null  float64
 11  V11     284807 non-null  float64
 12  V12     284807 non-null  float64
 13  V13     284807 non-null  float64
 14  V14     284807 non-null  float64
 15  V15     284807 non-null  float64
 16  V16     284807 non-null  float64
 17  V17     284807 non-null  float64
 18  V18     284807 non-null  float64
 19  V19     284807 non-null  float64
 20  V20     284807 non-null  float64
 21  V21     28

In [5]:
credit_data["Class"].value_counts()

Class
0    284315
1       492
Name: count, dtype: int64

In [6]:
legit=credit_data[credit_data.Class==0]
fraud=credit_data[credit_data.Class==1]

In [7]:
print(legit.shape)
print(fraud.shape)

(284315, 31)
(492, 31)


In [8]:
legit.Amount.describe()

count    284315.000000
mean         88.291022
std         250.105092
min           0.000000
25%           5.650000
50%          22.000000
75%          77.050000
max       25691.160000
Name: Amount, dtype: float64

In [9]:
fraud.Amount.describe()

count     492.000000
mean      122.211321
std       256.683288
min         0.000000
25%         1.000000
50%         9.250000
75%       105.890000
max      2125.870000
Name: Amount, dtype: float64

In [10]:
credit_data.groupby("Class").mean()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
Class,,,,,,,,,,,,,,,,,,,,,
0,94838.202258,0.008258,-0.006271,0.012171,-0.007860,0.005453,0.002419,0.009637,-0.000987,0.004467,...,-0.000644,-0.001235,-0.000024,0.000070,0.000182,-0.000072,-0.000089,-0.000295,-0.000131,88.291022
1,80746.806911,-4.771948,3.623778,-7.033281,4.542029,-3.151225,-1.397737,-5.568731,0.570636,-2.581123,...,0.372319,0.713588,0.014049,-0.040308,-0.105130,0.041449,0.051648,0.170575,0.075667,122.211321


Building a sample dataset containing similar legit and fraud transactions

In [11]:
legit_sample=legit.sample(n=492)

In [12]:
new=pd.concat([legit_sample,fraud],axis=0)

In [13]:
new.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
171718,120783.0,-0.853146,0.309548,1.656834,-0.933265,0.914678,1.657287,0.469822,0.212847,0.844270,...,0.184300,1.394718,-0.255885,-1.301253,-0.378992,0.711842,0.225623,-0.216051,38.78,0
28874,35208.0,0.924204,-0.595579,-0.241116,-0.687657,-0.474326,-0.210728,-0.229317,0.141167,1.356710,...,0.068737,0.119628,-0.262740,-0.392498,0.550115,-0.606014,0.069188,0.059207,143.77,0
268862,163415.0,0.121484,0.932940,-0.545011,-0.745542,1.132315,-0.237121,0.810265,0.115323,-0.203780,...,-0.314623,-0.808472,0.048673,0.102885,-0.436162,0.124102,0.218187,0.068752,1.98,0
246653,153268.0,1.922536,-0.632711,-0.210171,0.321032,-0.623886,0.461499,-1.032692,0.338195,1.209110,...,0.044271,0.127110,0.288229,0.332774,-0.536708,0.138786,-0.010150,-0.039514,26.38,0
186510,127140.0,0.044624,0.908268,0.309747,-0.588580,0.453493,-1.118614,1.048571,-0.239719,-0.129354,...,-0.253792,-0.523939,0.062958,0.054760,-0.471733,0.135880,0.251003,0.098677,2.67,0


In [14]:
new["Class"].value_counts()

Class
0    492
1    492
Name: count, dtype: int64

In [15]:
new.groupby("Class").mean()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
Class,,,,,,,,,,,,,,,,,,,,,
0,94489.516260,-0.008876,-0.001565,0.021211,0.045302,0.091941,0.050133,0.006562,0.009384,0.021076,...,-0.060484,0.039402,0.006565,0.030183,-0.028564,0.004789,0.024264,0.026547,-0.007490,82.457093
1,80746.806911,-4.771948,3.623778,-7.033281,4.542029,-3.151225,-1.397737,-5.568731,0.570636,-2.581123,...,0.372319,0.713588,0.014049,-0.040308,-0.105130,0.041449,0.051648,0.170575,0.075667,122.211321


In [16]:
X=new.drop(columns='Class',axis=1)
Y=new["Class"]

In [17]:
print(X)

            Time        V1        V2        V3        V4        V5        V6  \
171718  120783.0 -0.853146  0.309548  1.656834 -0.933265  0.914678  1.657287   
28874    35208.0  0.924204 -0.595579 -0.241116 -0.687657 -0.474326 -0.210728   
268862  163415.0  0.121484  0.932940 -0.545011 -0.745542  1.132315 -0.237121   
246653  153268.0  1.922536 -0.632711 -0.210171  0.321032 -0.623886  0.461499   
186510  127140.0  0.044624  0.908268  0.309747 -0.588580  0.453493 -1.118614   
...          ...       ...       ...       ...       ...       ...       ...   
279863  169142.0 -1.927883  1.125653 -4.518331  1.749293 -1.566487 -2.010494   
280143  169347.0  1.378559  1.289381 -5.004247  1.411850  0.442581 -1.326536   
280149  169351.0 -0.676143  1.126366 -2.213700  0.468308 -1.120541 -0.003346   
281144  169966.0 -3.113832  0.585864 -5.399730  1.817092 -0.840618 -2.943548   
281674  170348.0  1.991976  0.158476 -2.583441  0.408670  1.151147 -0.096695   

              V7        V8        V9  .

In [18]:
print(Y)

171718    0
28874     0
268862    0
246653    0
186510    0
         ..
279863    1
280143    1
280149    1
281144    1
281674    1
Name: Class, Length: 984, dtype: int64


In [19]:
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2,stratify=Y,random_state=2)

In [20]:
print(X.shape,X_train.shape,X_test.shape)

(984, 30) (787, 30) (197, 30)


In [21]:
model=LogisticRegression()

In [22]:
model.fit(X_train,Y_train)

LogisticRegression()

In [23]:
X_train_prediction=model.predict(X_train)
training_accuracy=accuracy_score(X_train_prediction,Y_train)

In [24]:
print("Accuracy of the training data:",training_accuracy)

Accuracy of the training data: 0.9377382465057179


In [25]:
X_test_prediction=model.predict(X_test)
testing_accuracy=accuracy_score(X_test_prediction,Y_test)

In [26]:
print("Accuracy of the testing data:",testing_accuracy)

Accuracy of the testing data: 0.9289340101522843
